In [25]:
import pandas as pd
from os import listdir
from os.path import isfile, join
import numpy as np

In [26]:
onlyfiles = [f for f in listdir('./ranking_sub18') if isfile(join('./ranking_sub18/', f))]
files = [i for i in onlyfiles if i.endswith('parquet') and ('ranking' in i)]
files_male = [i for i in files if 'Hombre' in i]
files_female = [i for i in files if 'Mujer' in i]

In [27]:
files_male

['ranking_5.000m Marcha_Hombre.parquet',
 'ranking_110m.v. (0,914)_Hombre.parquet',
 'ranking_4x100m_Hombre.parquet',
 'ranking_Decatlón Sub18_Hombre.parquet',
 'ranking_100m_Hombre.parquet',
 'ranking_10.000m Marcha_Hombre.parquet',
 'ranking_1.500m_Hombre.parquet',
 'ranking_Disco (1,5kg)_Hombre.parquet',
 'ranking_Peso (5kg)_Hombre.parquet',
 'ranking_Peso (3kg)_Hombre.parquet',
 'ranking_Pértiga_Hombre.parquet',
 'ranking_800m_Hombre.parquet',
 'ranking_Altura_Hombre.parquet',
 'ranking_3.000m_Hombre.parquet',
 'ranking_400m_Hombre.parquet',
 'ranking_200m_Hombre.parquet',
 'ranking_Martillo (5kg)_Hombre.parquet',
 'ranking_Jabalina (700g)_Hombre.parquet',
 'ranking_Longitud_Hombre.parquet',
 'ranking_400m.v. (0,84)_Hombre.parquet',
 'ranking_5.000m_Hombre.parquet',
 'ranking_Triple_Hombre.parquet',
 'ranking_2.000m Obst. 0,838m_Hombre.parquet']

In [28]:
# df = pd.read_parquet(files_male[0])
# df2 = pd.read_parquet(files_male[2])
# df = pd.concat([df, df2], axis=0, ignore_index=True)
# df

In [29]:
total_male = []
for file in files_male:
    df = pd.read_parquet(f'./ranking_sub18/{file}')
    total_male.append(df)

df_male = pd.concat(total_male, axis=0, ignore_index=True)
df_male['SEXO'] = 'Hombre'
df_male.shape


(2057, 16)

In [30]:

total_female = []
for file in files_female:
    df = pd.read_parquet(f'./ranking_sub18/{file}')
    total_female.append(df)

df_female = pd.concat(total_female, axis=0, ignore_index=True)
df_female['SEXO'] = 'Mujer'
df_female.shape

(2127, 16)

In [31]:
df = pd.concat([df_male, df_female], axis=0, ignore_index=True)
print(df.columns)
df.drop(columns=['Unnamed: 10', 'Unnamed: 11', 'RANK'], inplace=True)
df.head()

Index(['RANK', 'MARCA', 'ATLETA', 'CLUB', 'F.N.', 'PAÍS', 'FED.', 'POS.',
       'CIUDAD', 'FECHA', 'Unnamed: 10', 'PRUEBA', 'SECTOR', 'VIENTO',
       'Unnamed: 11', 'SEXO'],
      dtype='object')


,MARCA,ATLETA,CLUB,F.N.,PAÍS,FED.,POS.,CIUDAD,FECHA,PRUEBA,SECTOR,VIENTO,SEXO
0,20:39.93,DANIEL MONFORT MORENO,Playas de Castellon,25/01/2006,ESP,CVA,2,Cornellà de Llobregat,15/04/2023,5.000m Marcha,Marcha,NaN,Hombre
1,21:46.20,PABLO GONZALEZ VILLARES,At ADASAT Gran Canaria,03/05/2006,ESP,CNR,1,Las Palmas de Gran Canaria,13/05/2023,5.000m Marcha,Marcha,NaN,Hombre
2,22:05.40,JOAN QUEROL SERRANO,Valencia Club Atletismo,07/12/2006,ESP,CVA,1,Gandía,30/04/2023,5.000m Marcha,Marcha,NaN,Hombre
3,22:13.26,IVAN MOLINA ESPARTERO,CA Laietania,19/01/2006,ESP,CAT,2,Logroño,24/06/2023,5.000m Marcha,Marcha,NaN,Hombre
4,22:19.29,CESAR HIDALGO GARCIA,Juventud Guadix,11/02/2007,ESP,AND,1,Nerja,04/01/2023,5.000m Marcha,Marcha,NaN,Hombre


In [32]:
display(df[df['VIENTO'] == '-'])
df['VIENTO'] = df['VIENTO'].replace('-', np.nan)

,MARCA,ATLETA,CLUB,F.N.,PAÍS,FED.,POS.,CIUDAD,FECHA,PRUEBA,SECTOR,VIENTO,SEXO
355,11.24,LUIS TELLO RAMOS,Alcampo Scorpio 71,23/05/2006,ESP,ARA,3C1,Haslet - (USA),29/03/2023,100m,Velocidad,-,Hombre
1354,22.66,LUIS TELLO RAMOS,Alcampo Scorpio 71,23/05/2006,ESP,ARA,3C3,Justin - (USA),10/03/2023,200m,Velocidad,-,Hombre


In [33]:
df['MARCA'] = df['MARCA'].astype('str')
df['POS.'] = df['POS.'].astype('str')
def get_pos(x):
    if x == np.nan or x == '-':
        return np.nan
    pos = ''
    i = 0
    while i < len(x) and x[i].isdigit():
        pos += x[i]
        i += 1

    return int(pos)

df['POS.'] = df['POS.'].apply(get_pos)

df['VIENTO'] = df['VIENTO'].astype('float')

In [62]:
import re

def convertir_marca(marca):
    # Eliminar espacios en blanco al inicio y final de la cadena
    marca = marca.strip()

    # Si la marca está en el formato "mm:ss.sss"
    if re.match(r'^\d+:\d+\.\d+$', marca):
        minutos, segundos_centesimas = marca.split(':')
        segundos, centesimas = segundos_centesimas.split('.')
        return float(minutos) * 60 + float(segundos) + float(centesimas) / 100

    # Si la marca está en el formato "sss.sss"
    elif re.match(r'^\d+\.\d+$', marca):
        return float(marca)

    # Si la marca está en el formato "mm:ss"
    elif re.match(r'^\d+:\d+$', marca):
        minutos, segundos = marca.split(':')
        return float(minutos) * 60 + float(segundos)

    # Si no se puede determinar el formato, retornar None o lanzar una excepción según sea necesario
    else:
        return marca



In [67]:
df['MARCA_FLOAT'] = df['MARCA'].apply(convertir_marca)
df['MARCA_FLOAT'] = df['MARCA_FLOAT'].astype('float')

In [70]:
def get_year(fecha):
    if fecha == np.nan:
        return np.nan
    return fecha.split('/')[-1]

In [71]:
df['AÑO'] = df['F.N.'].apply(get_year)

In [72]:
df.to_parquet('sub18.parquet')